# December 05, 2024

https://adventofcode.com/2024/day/05

In [41]:
from collections import defaultdict
from itertools import chain

In [10]:
def parse_input( lines ):
    rules = list()
    updates = list()
    for line in lines:
        if len(line) == 0:
            continue
        if "|" in line:
            x,y = line.split("|")
            rules.append( [int(x) for x in line.split("|")] )
        else:
            updates.append( [int(x) for x in line.split(",")] )

    return rules, updates

In [11]:
test = f'''47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47'''

test = test.split("\n")
test = parse_input(test)

In [14]:
fn = "../data/2024/05.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz = [line.strip() for line in text]
puzz = parse_input(puzz)

# Part 1

In [23]:
def create_rule_dict( rules ):
    '''key = page; value = list of pages that can't follow it'''
    rule_dict = defaultdict(list)
    for rule in rules:
        rule_dict[rule[1]].append(rule[0])

    return rule_dict

In [25]:
nono = create_rule_dict(test[0])

In [26]:
nono

defaultdict(list,
            {53: [47, 75, 61, 97],
             13: [97, 61, 29, 47, 75, 53],
             61: [97, 47, 75],
             47: [97, 75],
             29: [75, 97, 53, 61, 47],
             75: [97]})

In [34]:
def check_update( update, nono ):
    # is there a better algo than n^2?
    for i, p0 in enumerate(update):
        for p1 in update[(i+1):]:
            if p1 in nono[p0]:
                return 0
    
    # valid ordering; return the middle page number
    return update[ int((len(update)-1)/2) ]

In [35]:
for up in test[1]:
    print(check_update(up, nono))

61
53
29
0
0
0


In [36]:
def part1( rules, updates ):
    nono = create_rule_dict( rules )

    tot = 0
    for up in updates:
        tot += check_update( up, nono )

    return tot

In [38]:
part1(*test)

143

In [39]:
part1(*puzz)

5091

# Part 2

In [93]:
def fix_order( update, nono ):
    fixed = [update[0]]

    # iterate over pages and make sure the follow all the rules
    for new_page in update[1:]:
        min_pos = 0
        max_pos = len(fixed)

        for pos, pg in enumerate(fixed):
            
            # check if this page can precede current page.
            if pg in nono[new_page]:
                # pg cannot follow new_page
                min_pos = max(min_pos, pos+1)
            if new_page in nono[pg]:
                # new_page cannot follow pg
                max_pos = min(max_pos, pos)

        assert min_pos <= max_pos

        fixed = fixed[:min_pos] + [new_page] + fixed[min_pos:]
    return fixed

def part2( rules, updates ):
    nono = create_rule_dict( rules )

    to_fix = list()
    for up in updates:
        if check_update( up, nono ) == 0:
            # this one doesn't work!
            to_fix.append( up )

    tot = 0
    for up in to_fix:
        fixed = fix_order( up, nono )
        tot += fixed[ int( (len(fixed)-1)/2) ]

    return tot


In [94]:
part2( *test )

123

In [95]:
part2(*puzz)

4681